In [2]:
# ===========================================
# 금융상품 추천 시스템 개발 (LightGBM)
# ===========================================

# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import warnings
import time
import joblib
warnings.filterwarnings('ignore')

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# LightGBM
try:
    from lightgbm import LGBMClassifier
    print("✅ LightGBM 라이브러리 import 성공!")
except ImportError as e:
    print(f"❌ Import 오류: {e}")
    print("LightGBM 설치: pip install lightgbm")

print("🚀 LightGBM 금융상품 추천 시스템")
print("=" * 50)

# 타겟 변수 이름
target_names = {
    'MMMF': '단기금융상품펀드',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

# ===========================================
# 📊 데이터 로드 및 전처리
# ===========================================
try:
    df = pd.read_csv('data/SCFP/cleaned_scf_data.csv')
    print(f"원본 데이터 로드 성공 : {len(df):,}개")
    
    # 1. 기본 데이터 정보 확인
    # print("\n📋 데이터 기본 정보:")
    # df.info()
    
    # 결측치 확인
    print(f"\n🔍 결측치 확인:")
    missing_counts = df.isnull().sum()
    if missing_counts.sum() == 0:
        print("✅ 결측치 없음")
    else:
        print(missing_counts[missing_counts > 0])
    
    # 4. 데이터 분할
    # 타겟 컬럼이 실제로 마지막 5개인지 확인 필요
    target_columns = list(target_names.keys())
    if all(col in df.columns for col in target_columns):
        X = df.drop(columns=target_columns)
        y = df[target_columns]
    else:
        # 기존 방식 유지
        X = df.iloc[:, :-5]
        y = df.iloc[:, -5:]
    
    print(f"\n✅ 전처리 완료: {X.shape[0]:,}명, {X.shape[1]}개 특성")
    
    # 5. 클래스 분포 확인
    print(f"\n📊 각 금융상품별 클래스 분포:")
    for target in target_names.keys():
        dist = y[target].value_counts()
        ratio = dist[1] / len(y) if 1 in dist else 0
        print(f"{target_names[target]}: 0={dist.get(0, 0)}, 1={dist.get(1, 0)} (비율: {ratio:.3f})")

except FileNotFoundError:
    print("※ 원본 데이터 로드 실패")
    # np.random.seed(42)
    # n_samples = 2000
    # X = pd.DataFrame(np.random.randn(n_samples, 14), 
    #                 columns=[f'feature_{i+1}' for i in range(14)])
    # y = pd.DataFrame(np.random.randint(0, 2, (n_samples, 5)),
    #                     columns=list(target_names.keys()))
    # print(f"✅ 샘플 데이터 생성: {X.shape[0]:,}명, {X.shape[1]}개 특성")

print(f"\n📦 데이터 준비 완료: X={X.shape}, y={y.shape}")

# ===========================================
# 🔀 데이터 분할
# ===========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
    # MultiLabel에서는 stratify 어려움
)
print(f"\n✂️  훈련 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")

# ===========================================
# 🤖 LightGBM 모델 설정
# ===========================================
# MultiOutputClassifier용 기본 모델 설정
base_lgbm_model = LGBMClassifier(
    # 복잡도 적당히 증가
    n_estimators=550,       # 450 → 550
    max_depth=7,           # 6 → 7 (깊이 증가)
    num_leaves=80,         # 60 → 80 (증가)
    
    # 학습률 조정
    learning_rate=0.045,   # 0.055 → 0.045 (더 세밀히)
    reg_alpha=0.5,         # 0.7 → 0.5 (정규화 완화)
    reg_lambda=1.2,        # 1.8 → 1.2 (정규화 완화)
    
    # 샘플링 증가
    subsample=0.82,        # 0.78 → 0.82
    colsample_bytree=0.85, # 0.78 → 0.85
    bagging_freq=5,        # 3 → 5
    
    # 제약 완화
    min_child_samples=18,  # 22 → 18
    min_split_gain=0.02,   # 0.03 → 0.02
    
    class_weight='balanced',
    objective='binary',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# MultiOutputClassifier로 래핑
multilabel_lgbm = MultiOutputClassifier(base_lgbm_model)

print(f"🔧 LightGBM MultiOutputClassifier 모델 설정 완료 (특성: {X.shape[1]}개)")

# ===========================================
# 🏃‍♂️ 모델 훈련
# ===========================================
# 훈련 부분 (기존 개별 모델 훈련 방식에서 변경)
print("\n🚀 MultiOutput LightGBM 훈련 시작...")
start_time = time.time()

multilabel_lgbm.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"✅ MultiOutput LightGBM 훈련 완료! (소요시간: {train_time:.1f}초)")

# ===========================================
# 🔮 예측 및 성능 평가
# ===========================================
print("\n🔍 예측 시작...")
test_pred = multilabel_lgbm.predict(X_test)
print("✅ 예측 완료!")

# 전체 성능 평가
accuracy = accuracy_score(y_test, test_pred)
precision = precision_score(y_test, test_pred, average='samples', zero_division=0)
recall = recall_score(y_test, test_pred, average='samples', zero_division=0)
f1 = f1_score(y_test, test_pred, average='samples', zero_division=0)

print("\n★ LightGBM 전체 모델 성능")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 점수:           {f1:.3f}")

✅ LightGBM 라이브러리 import 성공!
🚀 LightGBM 금융상품 추천 시스템
원본 데이터 로드 성공 : 22,975개

🔍 결측치 확인:
✅ 결측치 없음

✅ 전처리 완료: 22,975명, 11개 특성

📊 각 금융상품별 클래스 분포:
단기금융상품펀드: 0=22012, 1=963 (비율: 0.042)
양도성예금증서: 0=21183, 1=1792 (비율: 0.078)
비머니마켓펀드: 0=18371, 1=4604 (비율: 0.200)
주식보유: 0=16286, 1=6689 (비율: 0.291)
퇴직준비금유동성: 0=9398, 1=13577 (비율: 0.591)

📦 데이터 준비 완료: X=(22975, 11), y=(22975, 5)

✂️  훈련 데이터: (18380, 11), 테스트 데이터: (4595, 11)
🔧 LightGBM MultiOutputClassifier 모델 설정 완료 (특성: 11개)

🚀 MultiOutput LightGBM 훈련 시작...
✅ MultiOutput LightGBM 훈련 완료! (소요시간: 9.6초)

🔍 예측 시작...
✅ 예측 완료!

★ LightGBM 전체 모델 성능
정확도 (Accuracy):  0.676
정밀도 (Precision): 0.521
재현율 (Recall):    0.611
F1 점수:           0.547


In [3]:
# 훈련된 모델 저장
joblib.dump(multilabel_lgbm, 'multilabel_lgbm.joblib')
print("✅ 모델 저장 완료: multilabel_lgbm.joblib")

✅ 모델 저장 완료: multilabel_lgbm.joblib


In [3]:
# 훈련 데이터 예측
train_pred = multilabel_lgbm.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)  # 이 부분 수정!

print(f"훈련 정확도: {train_accuracy:.3f}")
print(f"테스트 정확도: {test_accuracy:.3f}")
print(f"차이: {train_accuracy - test_accuracy:.3f}")

훈련 정확도: 0.894
테스트 정확도: 0.827
차이: 0.067


In [4]:
# 1. 데이터 확인
print("훈련 데이터 모양:", X_train.shape, y_train.shape)
print("테스트 데이터 모양:", X_test.shape, y_test.shape)

# 2. 같은 모델로 예측 확인
train_pred_check = multilabel_lgbm.predict(X_train)
test_pred_check = multilabel_lgbm.predict(X_test)

print("훈련 예측 정확도:", accuracy_score(y_train, train_pred_check))
print("테스트 예측 정확도:", accuracy_score(y_test, test_pred_check))

# 3. 타겟 분포 확인
print("훈련 타겟 분포:")
print(y_train.mean())
print("테스트 타겟 분포:")
print(y_test.mean())

훈련 데이터 모양: (18380, 14) (18380, 5)
테스트 데이터 모양: (4595, 14) (4595, 5)
훈련 예측 정확도: 0.8942872687704027
테스트 예측 정확도: 0.8269858541893362
훈련 타겟 분포:
MMMF       0.041893
CDS        0.077476
NMMF       0.201251
STOCKS     0.290860
RETQLIQ    0.591023
dtype: float64
테스트 타겟 분포:
MMMF       0.042002
CDS        0.080087
NMMF       0.196953
STOCKS     0.292274
RETQLIQ    0.590642
dtype: float64
